In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

2025-11-15 18:51:13.127772: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-15 18:51:13.158613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-15 18:51:13.827115: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def extract_surface_lines_np(brep_str):
    lines = brep_str.strip().split('\n')
    surface_stop = 0
    surface_line = 0

    for i, line in enumerate(lines):
        if line.startswith('Surface'):
            surface_line = i
        if line.startswith('Triangulations'):
            surface_stop = i
            break

    surfaces = []

    for i in range(surface_line + 1, surface_stop):
        surfaces.append(lines[i].strip().split(' '))

    surfaces_flat = []
    for i in surfaces:
        for j in i:
            surfaces_flat.append(float(j))

    return np.array(surfaces_flat, dtype=np.float32)

In [4]:
import os

shapes = ['cone', 'cube', 'cylinder', 'sphere', 'pipe', 'round_plate', 'rectangular_plate']

file_list = [os.listdir('models_2/' + shape) for shape in shapes]

X = []
y = []

for shape in shapes:
    y.append([shape]*len(os.listdir('models_2/' + shape)))

# flatten
file_list = [item for items in file_list for item in items]
y = [item for items in y for item in items]

for i in range(0, len(file_list)):
    with open('models_2/' + y[i] + '/' + file_list[i], 'r', encoding='utf-8') as f:
        brep_str = f.read()
        X.append(extract_surface_lines_np(brep_str))

In [5]:
from numpy import float32
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

maxlen = 1024
X_pad = pad_sequences(X, maxlen=maxlen, dtype='float32', padding='post', truncating='post', value=0.0)
X_pad = X_pad[..., None]

num_outputs = len(shapes)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_arr = np.asarray(y_encoded, dtype='int32')

X_train, X_test, y_train, y_test = train_test_split(X_pad, y_arr, test_size=0.2, random_state=3)

model = models.Sequential([
    layers.Input(shape=(maxlen, 1)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_outputs, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

W0000 00:00:1763250700.728460  241790 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
model_trained = model.fit(X_train, y_train, epochs=10, validation_split=0.1, batch_size=64)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4345 - loss: 2.4977 - val_accuracy: 0.7500 - val_loss: 0.7806
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9385 - loss: 0.7922 - val_accuracy: 1.0000 - val_loss: 0.5204
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.5577 - val_accuracy: 1.0000 - val_loss: 0.3677
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.3961 - val_accuracy: 1.0000 - val_loss: 0.2692
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.2847 - val_accuracy: 1.0000 - val_loss: 0.2037
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.2083 - val_accuracy: 1.0000 - val_loss: 0.1566
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.1560 - val_accuracy: 1.0000 - val_loss: 0.1194
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.1209 - val_accuracy: 1.0000 - val_loss: 0.0953
Epoch 9